In [0]:
SELECT current_catalog() AS catalog, current_schema() AS schema;

In [0]:
SHOW CATALOGS;

In [0]:
SHOW SCHEMAS IN workspace;


In [0]:
CREATE SCHEMA IF NOT EXISTS workspace.bronze;


In [0]:
SHOW TABLES IN workspace.default;


In [0]:
ALTER TABLE workspace.default.device_messages
RENAME TO workspace.bronze.device_messages_raw;

ALTER TABLE workspace.default.rapid_step_tests
RENAME TO workspace.bronze.rapid_step_tests_raw;


In [0]:
SHOW TABLES IN workspace.bronze;


In [0]:
SELECT 'device_messages_raw' AS table_name, COUNT(*) AS rows
FROM workspace.bronze.device_messages_raw
UNION ALL
SELECT 'rapid_step_tests_raw', COUNT(*)
FROM workspace.bronze.rapid_step_tests_raw;

In [0]:
SELECT
  device_id,
  COUNT(*) AS message_count
FROM workspace.bronze.device_messages_raw
GROUP BY device_id
ORDER BY message_count DESC;


In [0]:
DESCRIBE TABLE workspace.bronze.rapid_step_tests_raw;


In [0]:
SELECT
  customer,
  device_id,
  to_timestamp(from_unixtime(start_time / 1000)) AS start_ts,
  to_timestamp(from_unixtime(stop_time / 1000))  AS stop_ts,
  to_timestamp(from_unixtime(test_time / 1000))  AS test_ts,
  total_steps,
  size(step_points) AS step_points_len
FROM workspace.bronze.rapid_step_tests_raw
LIMIT 20;


In [0]:
SELECT
  customer,
  device_id,
  to_timestamp(from_unixtime(start_time / 1000)) AS start_ts,
  to_timestamp(from_unixtime(stop_time / 1000))  AS stop_ts,
  test_time AS test_time_raw,
  total_steps,
  size(step_points) AS step_points_len
FROM workspace.bronze.rapid_step_tests_raw
LIMIT 20;


In [0]:
CREATE OR REPLACE VIEW workspace.bronze.rapid_step_tests_clean AS
SELECT
  customer,
  device_id,
  to_timestamp(from_unixtime(start_time / 1000)) AS start_ts,
  to_timestamp(from_unixtime(stop_time / 1000))  AS stop_ts,
  test_time                                     AS test_time_raw,
  step_points,
  total_steps,
  token
FROM workspace.bronze.rapid_step_tests_raw;


In [0]:
DESCRIBE TABLE workspace.bronze.device_messages_raw;

In [0]:
SELECT *
FROM workspace.bronze.device_messages_raw
LIMIT 10;

In [0]:
SELECT
  device_id,
  date,
  to_timestamp(from_unixtime(date / 1000))      AS date_ts_guess,
  timestamp,
  to_timestamp(from_unixtime(timestamp / 1000)) AS ts_ts_guess,
  message
FROM workspace.bronze.device_messages_raw
LIMIT 25;

In [0]:
CREATE OR REPLACE VIEW workspace.bronze.device_messages_clean AS
SELECT
  message_origin,
  session_key,
  message,
  device_id,
  sensor_type,
  distance,
  date AS date_raw,
  timestamp AS timestamp_raw,
  to_timestamp(from_unixtime(date / 1000)) AS event_ts
FROM workspace.bronze.device_messages_raw;


In [0]:
SELECT device_id, event_ts, message, sensor_type, distance
FROM workspace.bronze.device_messages_clean
ORDER BY event_ts DESC
LIMIT 20;


In [0]:
SELECT
  r.customer,
  r.device_id,
  r.start_ts,
  r.stop_ts,
  r.total_steps,
  d.event_ts,
  d.sensor_type,
  d.distance,
  d.message
FROM workspace.bronze.rapid_step_tests_clean r
JOIN workspace.bronze.device_messages_clean d
  ON r.device_id = d.device_id
 AND d.event_ts BETWEEN r.start_ts AND r.stop_ts
LIMIT 50;


In [0]:
CREATE OR REPLACE VIEW workspace.bronze.step_test_device_messages AS
SELECT
  r.customer,
  r.device_id,
  r.start_ts,
  r.stop_ts,
  r.total_steps,
  r.step_points,
  d.event_ts,
  d.sensor_type,
  d.distance,
  d.message_origin,
  d.session_key,
  d.message
FROM workspace.bronze.rapid_step_tests_clean r
JOIN workspace.bronze.device_messages_clean d
  ON r.device_id = d.device_id
 AND d.event_ts BETWEEN r.start_ts AND r.stop_ts;


In [0]:
SHOW TABLES IN workspace.bronze;

In [0]:
CREATE OR REPLACE VIEW workspace.bronze.raw_device_message AS
SELECT * FROM workspace.bronze.device_messages_raw;

CREATE OR REPLACE VIEW workspace.bronze.raw_rapid_step_test AS
SELECT * FROM workspace.bronze.rapid_step_tests_raw;


In [0]:
SHOW TABLES IN workspace.bronze;


In [0]:
DESCRIBE TABLE workspace.bronze.raw_device_message;
SELECT * FROM workspace.bronze.raw_device_message LIMIT 10;

DESCRIBE TABLE workspace.bronze.raw_rapid_step_test;
SELECT * FROM workspace.bronze.raw_rapid_step_test LIMIT 10;


For workspace.bronze.raw_device_message: This table stores raw, high-frequency sensor messages emitted by each STEDI device. Each row captures a device identifier, a recorded time (epoch milliseconds), the sensor type, and the raw measured distance (plus message/session metadata). This matters for ML because these timestamped sensor readings become the core feature signals (distance over time, sensor patterns) that can be aligned to labeled events like step tests.

For workspace.bronze.raw_rapid_step_test: This table stores raw step-test sessions for a device, including the device_id, the start and stop times of the test window (epoch milliseconds), and step-related outputs like total_steps and step_points. This matters for ML because it provides the “ground truth” test windows used to label sensor readings as occurring during a step test versus outside it, enabling supervised learning.

In [0]:
CREATE SCHEMA IF NOT EXISTS workspace.silver;


In [0]:
CREATE TABLE IF NOT EXISTS workspace.silver.labeled_step_test
USING DELTA
AS
SELECT
  d.date AS timestamp,
  d.device_id AS deviceId,
  d.sensor_type AS sensorType,
  CAST(regexp_replace(d.distance, '[^0-9]', '') AS INT) AS distance_cm,
  CASE
    WHEN to_timestamp(from_unixtime(d.date / 1000))
         BETWEEN to_timestamp(from_unixtime(r.start_time / 1000))
             AND to_timestamp(from_unixtime(r.stop_time / 1000))
      THEN 'step'
    ELSE 'no_step'
  END AS step_label,
  'device' AS source,
  r.start_time AS startTime,
  r.stop_time AS stopTime,
  concat(d.device_id, '_', cast(r.start_time as string), '_', cast(r.stop_time as string)) AS testId
FROM workspace.bronze.device_messages_raw d
LEFT JOIN workspace.bronze.rapid_step_tests_raw r
  ON d.device_id = r.device_id;


In [0]:
SELECT COUNT(*) AS rows
FROM workspace.silver.labeled_step_test;

SELECT *
FROM workspace.silver.labeled_step_test
LIMIT 20;


Silver table schema (Unity Catalog)

In [0]:
SELECT 
  column_name AS `Column Name`, 
  data_type AS `Data Type`,
  comment AS Comment
FROM workspace.information_schema.columns
WHERE table_schema = 'silver'
  AND table_name = 'labeled_step_test'
ORDER BY ordinal_position;
